# Short Circuit

Let's see how we can make a short circuit calculation.

In [7]:
import getpass

import numpy as np

from roseau.load_flow import *

We will start by creating a simple network composed of two LV lines.
As usual with short-circuit calculations, we won't add any loads.

<div class="alert alert-info">

**Note:**
While impedance and currents loads could technically be added to the network, it is not possible to add a power load on the same bus as the one we want to compute the short-circuit on. This is because having `S = UI*` with `U=0` is impossible.

</div>

In [8]:
# Create three buses
source_bus = Bus(id="sb", phases="abcn")
bus1 = Bus(id="b1", phases="abcn")
bus2 = Bus(id="b2", phases="abcn")

# Define the reference of potentials to be the neutral of the source bus
ground = Ground(id="gnd")
pref = PotentialRef(id="pref", element=ground)  # Fix the potential of the ground at 0 V
ground.connect(bus=source_bus)

# Create a LV source at the first bus
un = 400 / np.sqrt(3)  # Volts (phase-to-neutral because the source is connected to the neutral)
source_voltages = [un, un * np.exp(-2j * np.pi / 3), un * np.exp(2j * np.pi / 3)]
vs = VoltageSource(id="vs", bus=source_bus, phases="abcn", voltages=source_voltages)

# Add LV lines
lp1 = LineParameters.from_name_lv("S_AL_240")
line1 = Line(id="line1", bus1=source_bus, bus2=bus1, parameters=lp1, length=1.0, ground=ground)
lp2 = LineParameters.from_name_lv("S_AL_150")
line2 = Line(id="line2", bus1=bus1, bus2=bus2, parameters=lp2, length=2.0, ground=ground)

# Create network
en = ElectricalNetwork.from_element(source_bus)

We can now add a short circuit. Let's first make a phase-to-phase short circuit:

In [3]:
bus2.short_circuit("a", "b")

Let's run the load flow, and get the current results.

In [4]:
# Authentication
auth = (input("Please enter your username:"), getpass.getpass("Please enter your password:"))

In [6]:
en.solve_load_flow(auth=auth)
en.res_branches

current1                    current2  \
branch_id phase                                                      
line1     a      376.726141+75.273614j -3.765079e+02-7.516544e+01j   
          b     -376.137057-74.958399j  3.761182e+02+7.496233e+01j   
          c      -0.4933660-0.4185120j  4.929085e-01+2.055072e-01j   
          n      -0.0957190+0.1032970j -1.031467e-01-2.394060e-03j   
line2     a      376.507942+75.165439j -3.764496e+02-7.493254e+01j   
          b     -376.118181-74.962326j  3.764496e+02+7.493254e+01j   
          c      -0.4929090-0.2055070j -1.421085e-14+1.136868e-13j   
          n       0.1031470+0.0023940j  4.263256e-14+0.000000e+00j   

                                     power1                      power2  \
branch_id phase                                                           
line1     a      87001.175581-17383.696547j -6.962717e+04+2.413923e+04j   
          b      58424.246079+66571.961003j -4.114025e+04-5.981005e+04j   
          c      -26.733437-00146.99870000j -1.490257e+01+1.267102e+02j   
          n      0.0000000000+0.0000000000j -1.488217e-01+8.492953e-01j   
line2     a      69627.173712-24139.231256j -1.421789e+04+4.199279e+04j   
          b      41140.247738+59810.047467j  1.421789e+04-4.199279e+04j   
          c       14.902575-00126.71022600j  2.725799e-11+1.047715e-11j   
          n      0.1488220000-0.8492950000j  2.078778e-13-1.048707e-12j   

                             potential1              potential2  
branch_id phase                                                  
line1     a      230.9401008-0.0000000j   190.149827-26.152266j  
          b     -115.470054-200.000000j  -74.719733-173.911328j  
          c     -115.470054+200.000000j -117.063139+208.259546j  
          n      0.00000000+0.00000000j  1.63304500-8.19595200j  
line2     a       190.149827-26.152266j   57.686778-100.066962j  
          b      -74.719733-173.911328j   57.686778-100.066962j  
          c     -117.063139+208.259546j -120.249589+224.732658j  
          n      1.63304500-8.19595200j   4.876033-024.5987340j

We can also make a multi-phase short circuit, by first removing the current short adding and making another one.

In [8]:
bus2.remove_short_circuit()
bus2.short_circuit("a", "b", "c")
en.solve_load_flow(auth=auth)
en.res_branches

current1                    current2  \
branch_id phase                                                       
line1     a      371.737757-146.296806j -3.715461e+02+1.463893e+02j   
          b     -325.125461-309.415778j  3.251080e+02+3.094177e+02j   
          c      -46.494657+455.595154j  4.650593e+01-4.557992e+02j   
          n     -0.11763900+0.11743000j -6.777239e-02-7.800686e-03j   
line2     a      371.546147-146.389264j -3.715851e+02+1.465620e+02j   
          b     -325.107990-309.417735j  3.252783e+02+3.093020e+02j   
          c      -46.505929+455.799198j  4.630677e+01-4.558639e+02j   
          n      0.06777200+0.00780100j  7.105427e-15+4.263256e-14j   

                                     power1                      power2  \
branch_id phase                                                           
line1     a      85849.157555+33785.800020j -6.352586e+04-2.464708e+04j   
          b      99425.410149+29296.835634j -7.575517e+04-2.003848e+04j   
          c      96487.771322+43308.665615j -7.540992e+04-3.185850e+04j   
          n      0.0000000000+0.0000000000j -4.016327e-01+5.328110e-01j   
line2     a      63525.864916+24647.076142j  3.541552e+03-1.646579e+03j   
          b      75755.171610+20038.477314j  1.413844e+00+4.388761e+03j   
          c      75409.916329+31858.500204j -3.542966e+03-2.742182e+03j   
          n      0.4016330000-0.5328110000j -7.629415e-13-1.012519e-12j   

                             potential1             potential2  
branch_id phase                                                 
line1     a      230.9401008+0.0000000j   170.626282-0.890279j  
          b     -115.470054-200.000000j -91.485558-148.706685j  
          c     -115.470054+200.000000j -85.882505+156.682747j  
          n      0.00000000+0.00000000j   6.7417810-7.0857840j  
line2     a      170.6262802-0.8902790j  -6.7353330+7.0878040j  
          b      -91.485558-148.706685j  -6.7353330+7.0878040j  
          c      -85.882505+156.682747j  -6.7353330+7.0878040j  
          n      6.74178100-7.08578400j   20.205999-21.263413j

Finally, we can make a phase-to-ground short-circuit.

In [9]:
bus2.remove_short_circuit()
bus2.short_circuit("a", ground=ground)
en.solve_load_flow()
en.res_branches

current1                    current2  \
branch_id phase                                                      
line1     a      96.012654-188.549830j -9.579721e+01+1.886470e+02j   
          b       0.5278520-0.4176090j -5.447323e-01+4.163191e-01j   
          c      -0.4126860-0.5103670j  4.341498e-01+2.786264e-01j   
          n      -0.0396740-0.0650550j -1.739503e-01+1.803321e-01j   
line2     a      95.797205-188.647037j -9.590970e+01+1.889045e+02j   
          b       0.5447320-0.4163190j  0.000000e+00+0.000000e+00j   
          c      -0.4341500-0.2786260j  1.421085e-14+0.000000e+00j   
          n       0.1739500-0.1803320j  5.684342e-14+9.947598e-14j   

                                     power1                      power2  \
branch_id phase                                                           
line1     a      22173.172540+43543.718061j -1.685871e+04-2.947666e+04j   
          b       22.570709-00153.79182900j -3.386010e+00+1.921577e+02j   
          c      -54.420413-00141.46933500j -2.119180e+01+1.217538e+02j   
          n      0.0000000000+0.0000000000j  4.190759e+00+1.361504e+01j   
line2     a      16858.712030+29476.655117j  0.000000e+00+0.000000e+00j   
          b      3.386010-000192.157653000j  0.000000e+00+0.000000e+00j   
          c       21.191795-00121.75379700j -3.625336e-12+1.837655e-12j   
          n      -4.190759-00013.615042000j -1.630952e-11+1.075219e-11j   

                             potential1              potential2  
branch_id phase                                                  
line1     a      230.9401008-0.0000000j  160.2978106-7.9652950j  
          b     -115.470054-200.000000j -166.266602-225.684569j  
          c     -115.470054+200.000000j -162.048885+176.443020j  
          n      0.00000000+0.00000000j -50.7214930-25.6873780j  
line2     a      160.2978106-7.9652950j  0.00000000+0.00000000j  
          b     -166.266602-225.684569j -267.742704-277.018629j  
          c     -162.048885+176.443020j -255.110334+129.313498j  
          n     -50.7214930-25.6873780j  -152.108420-77.035237j